In [1]:
import os
import struct
import numpy as np
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
    with open(images_path, 'rb') as imgpath: 
        magic, num, rows, cols = struct.unpack(">IIII",
                                               imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
    return images, labels

In [2]:
X_train, y_train = load_mnist('./downloads/mnist', kind='train')
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))

X_test, y_test = load_mnist('./downloads/mnist', kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

Rows: 60000, columns: 784
Rows: 10000, columns: 784


In [3]:
from keras.utils import np_utils
print('First 3 labels: ', y_train[:3])
y_train_ohe = np_utils.to_categorical(y_train)
print('\nFirst 3 labels (one-hot):\n', y_train_ohe[:3])

Using TensorFlow backend.


First 3 labels:  [5 0 4]

First 3 labels (one-hot):
 [[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]


In [4]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

np.random.seed(1)
model = Sequential()
model.add(Dense(input_dim=X_train.shape[1],
                units=50,
                kernel_initializer='uniform',
                activation='tanh'))
model.add(Dense(input_dim=50,
                units=50,
                kernel_initializer='uniform',
                activation='tanh'))
model.add(Dense(input_dim=50,
                units=y_train_ohe.shape[1],
                kernel_initializer='uniform',
                activation='softmax'))
sgd = SGD(lr=0.001, decay=1e-7, momentum=.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [5]:
model.fit(X_train,
          y_train_ohe,
          epochs=50,
          batch_size=300,
          verbose=1,
          validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 3s - loss: 2.2204 - acc: 0.4277 - val_loss: 2.0852 - val_acc: 0.6235
Epoch 2/50
54000/54000 [==============================] - 4s - loss: 1.8277 - acc: 0.6205 - val_loss: 1.5049 - val_acc: 0.6582
Epoch 3/50
54000/54000 [==============================] - 3s - loss: 1.2982 - acc: 0.6578 - val_loss: 1.0738 - val_acc: 0.7292
Epoch 4/50
54000/54000 [==============================] - 3s - loss: 0.9787 - acc: 0.7466 - val_loss: 0.8149 - val_acc: 0.8128
Epoch 5/50
54000/54000 [==============================] - 3s - loss: 0.7767 - acc: 0.8124 - val_loss: 0.6518 - val_acc: 0.8555
Epoch 6/50
54000/54000 [==============================] - 3s - loss: 0.6430 - acc: 0.8490 - val_loss: 0.5457 - val_acc: 0.8847
Epoch 7/50
54000/54000 [==============================] - 3s - loss: 0.5549 - acc: 0.8678 - val_loss: 0.4694 - val_acc: 0.8927
Epoch 8/50
54000/54000 [==============================] - 3s -

In [6]:
y_train_pred = model.predict_classes(X_train, verbose=0)
print('First 3 predictions: ', y_train_pred[:3])

train_acc = np.sum(y_train == y_train_pred, axis=0) / X_train.shape[0]
print('Training accuracy: %.2f%%' % (train_acc * 100))

y_test_pred = model.predict_classes(X_test, verbose=0)
test_acc = np.sum(y_test == y_test_pred, axis=0) / X_test.shape[0]
print('Test accuracy: %.2f%%' % (test_acc * 100))

First 3 predictions:  [5 0 4]
Training accuracy: 94.30%
Test accuracy: 93.79%
